# Generating object masks from input prompts with SAM
This notebook shows how to generate object masks from input prompts with the Segment Anything Model (SAM). Make sure you use GPU runtime for this notebook. 

## Setup

In [1]:
# Import the os module
import os

path = 'tmp/'

try:
    os.chdir(path)
    print("Current working directory: {0}".format(os.getcwd()))
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

Current working directory: /home/ec2-user/SageMaker/ODP_Demo/tmp


## Import Libaray

In [2]:
import leafmap
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [3]:
m = leafmap.Map(center=[37.6412, -122.1353], zoom=15, height="800px")
m.add_basemap("SATELLITE")
m.attribution_control=False
m

Map(center=[37.6412, -122.1353], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [4]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-122.1497, 37.6311, -122.1203, 37.6458]

In [5]:
image = "satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=16, source="Satellite", overwrite=True)

Downloaded image 01/30
Downloaded image 02/30
Downloaded image 03/30
Downloaded image 04/30
Downloaded image 05/30
Downloaded image 06/30
Downloaded image 07/30
Downloaded image 08/30
Downloaded image 09/30
Downloaded image 10/30
Downloaded image 11/30
Downloaded image 12/30
Downloaded image 13/30
Downloaded image 14/30
Downloaded image 15/30
Downloaded image 16/30
Downloaded image 17/30
Downloaded image 18/30
Downloaded image 19/30
Downloaded image 20/30
Downloaded image 21/30
Downloaded image 22/30
Downloaded image 23/30
Downloaded image 24/30
Downloaded image 25/30
Downloaded image 26/30
Downloaded image 27/30
Downloaded image 28/30
Downloaded image 29/30
Downloaded image 30/30
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [6]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [7]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(center=[37.6412, -122.1353], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
checkpoint = os.path.join(out_dir, "sam_vit_h_4b8939.pth")

Set `automatic=False` to disable the `SamAutomaticMaskGenerator` and enable the `SamPredictor`.

In [10]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    automatic=False,
    sam_kwargs=None,
)

Specify the image to segment. 

In [11]:
sam.set_image(image)

## Image segmentation with input points

A single point can be used to segment an object. The point can be specified as a tuple of (x, y), such as (col, row) or (lon, lat). The points can also be specified as a file path to a vector dataset. For non (col, row) input points, specify the `point_crs` parameter, which will automatically transform the points to the image column and row coordinates.

Try a single point input:

In [12]:
point_coords = [[-122.1419, 37.6383]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output='mask1.tif')
m.add_raster('mask1.tif', layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
m

Map(center=[37.6412, -122.1353], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

Try multiple points input:

In [13]:
point_coords = [[-122.1464, 37.6431], [-122.1449, 37.6415], [-122.1451, 37.6395]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output='mask2.tif')
m.add_raster('mask2.tif', layer_name="Mask2", nodata=0, cmap="Greens", opacity=1)
m

Map(center=[37.6412, -122.1353], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## Interactive segmentation

Display the interactive map and use the marker tool to draw points on the map. Then click on the `Segment` button to segment the objects. The results will be added to the map automatically. Click on the `Reset` button to clear the points and the results.

In [15]:
m = sam.show_map()
m.attribution_control=False
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Outputs Reference: 
Reference in GIF format pulled from imgur.com.

![](https://i.imgur.com/2Nyg9uW.gif)